In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mockdata/final_data1.pickle


In [1]:
import os
os.chdir(r'/kaggle/working')


In [1]:
import re
import os

import random
import torch.optim as optim
import torch

import re
#
from torchtext.data.metrics import bleu_score
from rouge import Rouge

import pickle
from transformers import T5ForConditionalGeneration, T5Tokenizer
file=open('/kaggle/input/mockdata/final_data1.pickle','rb')
data=pickle.load(file)
random.shuffle(data)



In [2]:
def preprocess_article(article):
    article = re.sub(r"http\S+", "", article)
    article = re.sub(r"www.\S+", "", article)
    article = re.sub(r"<\S+", "", article)
    #article = re.sub('[.]\n?', ". ",article)
    #article = re.sub(r'[^\w\s]', '', article) 
    article = re.sub('\n+', " ",article)
    article = article.strip()
    #if len(article) > 60:
    #    article = article[:-60]
    return article
#print(preprocess_article(t))

['hindi news happylife if you want to reduce the risk of cancer, drink green tea, it increases the protein in the body that kills cancer cells. Fed up with ads? Install for news without ads Dainik Bhaskar App New Research: If you want to reduce the risk of cancer, drink green-tea, it increases such protein in the body.\n',
 'New research: If you want to reduce the risk of cancer, drink green-tea, it increases the protein in the body that kills the cancer cells.\n',
 '0']

In [4]:

for line in data:
        line[0]= preprocess_article(line[0].lstrip().rstrip().lower())
        line[1]= preprocess_article(line[1].lstrip().rstrip().lower())
data_train=data[:3000]
data_val=data[3000:]


model=T5ForConditionalGeneration.from_pretrained('t5-base').cuda()
tokenizer=T5Tokenizer.from_pretrained('t5-base')
rouge1=Rouge()

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

In [ ]:
def rouge(sentence,ref):
    s=''
    flag=0
    for char in sentence:
        if char=='<':
            flag=1
        elif char=='>':
            flag=0
        elif flag==0:
            s+=char

    gen=re.sub('\s+',' ',s).rstrip().lstrip()
#     print(gen)
    if gen=='':
        return 0,0,0
    rg=rouge1.get_scores(gen,ref)
    r1,r2,rl=rg[0]["rouge-1"]['f'], \
    rg[0]["rouge-2"]['f'],rg[0]["rouge-l"]['f']
    
    return r1,r2,rl

def generate_batch(data):
    output=random.sample(data,4)

    inp,label=[],[]
    for dat in output:
            inp.append(dat[0])
            label.append(dat[1])
            
    return inp,label
def val(dev_data):

    r1_,r2_,rl_=0,0,0
    candidate_corpus,references_corpus=[],[]

    for line in dev_data:
        inp,label=[line[0]],[line[1]]
        input=tokenizer.prepare_seq2seq_batch(src_texts=inp,
                                              tgt_texts=label, padding=True, return_tensors='pt')
        
        output=model.generate(input_ids=input['input_ids'].cuda(),
                              num_beams=5, early_stopping=True, max_length=20)
        out=tokenizer.batch_decode(output)
        out[0] = re.sub(r"<\S+", "", out[0])
        
        candidate_corpus.append(tokenizer.tokenize(out[0]))
        references_corpus.append([tokenizer.tokenize(label[0])])

        r1,r2,rl=rouge(out[0],line[1])
        r1_+=r1
        r2_+=r2
        rl_+=rl

    r1_/=(len(dev_data)*0.01)
    r2_/=(len(dev_data)*0.01)
    rl_/=(len(dev_data)*0.01)
    bleu=0
    bleu=100*bleu_score(candidate_corpus, references_corpus)

    return r1_,r2_,rl_,bleu


In [6]:


optimizer=optim.AdamW(model.parameters(),lr=0.00002)

scalar=0
val_score=0
for i in range(2000):
        model.train()
        inp,label=generate_batch(data_train)
        input=tokenizer.prepare_seq2seq_batch(src_texts=inp, tgt_texts=label, padding=True, return_tensors='pt',max_length=600,truncation=True)
        outputs=model(input_ids=input['input_ids'].cuda(),labels=input['labels'].cuda())
        loss=outputs[0]

        scalar+=loss.item()
        if(i+1)%50==0:
                print('iteration={}, training loss={}'.format(i+1,scalar/(4*50)))
                scalar=0
        if(i+1)%2000==0:
                r1,r2,rl,bleu=val(data_val)
                print('validation BLEU={}, validation R1={}, validation R2={}, validation RL={}'.format(bleu,r1,r2,rl))
                
                

        loss.backward()
        optimizer.step()

iteration=50, training loss=1.0169041705131532
iteration=100, training loss=0.6731205302476883
iteration=150, training loss=0.6208210551738739
iteration=200, training loss=0.5606093782186509
iteration=250, training loss=0.5213078415393829
iteration=300, training loss=0.48459365040063856
iteration=350, training loss=0.5068568247556686
iteration=400, training loss=0.45633628994226455
iteration=450, training loss=0.47629580825567247
iteration=500, training loss=0.44122555434703825
iteration=550, training loss=0.4272786337137222
iteration=600, training loss=0.460400798022747
iteration=650, training loss=0.45351127684116366
iteration=700, training loss=0.4632643106579781
iteration=750, training loss=0.4120591467618942
iteration=800, training loss=0.4207980468869209
iteration=850, training loss=0.4307216280698776
iteration=900, training loss=0.41163205742836
iteration=950, training loss=0.42135584145784377
iteration=1000, training loss=0.4236678838729858
iteration=1050, training loss=0.37826

In [5]:
!pip install -U sentence-transformers
!pip install -U rouge

     |████████████████████████████████| 64 kB 1.8 MB/s eta 0:00:011
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-py3-none-any.whl size=103066 sha256=ee2da78711cb40e3e64cc4af95a3e961b5460b6a49d30c3438c50b103d2c8b44
  Stored in directory: /root/.cache/pip/wheels/1c/a2/db/5d9e7c4aa8dbd82718c202fb1bc7118c2d3bf0925af92943f2
Successfully built sentence-transformers


In [18]:
'''
data_val = data[3000:3500]
r1,r2,rl,bleu=val(data_val)
print('validation BLEU={}, validation R1={}, validation R2={}, validation RL={}'.format(bleu,r1,r2,rl))
'''

validation BLEU=22.70092736806129, validation R1=33.47478494404291, validation R2=21.31588646212235, validation RL=32.48059324320051


In [14]:
data_x = []
for row in data_val:
    
    if row[2]=='1':
        data_x.append(row)   

In [15]:
len(data_x)


534

## MOBILE THEME VALIDATION ARTICLES

In [17]:
r1,r2,rl,bleu=val(data_x)
print('validation BLEU={}, validation R1={}, validation R2={}, validation RL={}'.format(bleu,r1,r2,rl))

validation BLEU=23.32314254925191, validation R1=29.971884569561293, validation R2=16.572101959069425, validation RL=30.019844323844747


In [7]:
torch.save(model.state_dict(),'modelt5_.pth')